# Why is it worth it?

Our world is **highly dynamic** and even if we find periodic patterns we can't assume that they will stay there forever. When we just train models on data that was collected in the past and apply them only for inference on future data we will likely experience a break-down of our evaluation criteria someday. Especially the pandemic of covid-19 should have opened the eyes that **stochastic events can cause changes in the overall system that are not easy to foresee and/or somehow irrational. In such cases machine learning models that are not able to adapt to environmental changes like a chameleon are doomed to fail.**


With this notebook I like to learn more about evolutionary algorithms and share my experiences with the community. If you like to play with code and ideas, just fork. ;-)

Happy kaggling!


<img src="https://cdn.pixabay.com/photo/2020/01/10/21/07/terrarium-4756280_1280.jpg" width="900px">

## Table of contents

1. [What is Differential Evolution?](#differential_evolution)
2. [How does it work?](#how)
3. [Code to built the model](#de_model)
4. [Some simple examples](#simple_examples)
5. [Prepare for the Titanic](#prepare_titanic)
6. [Build the NNs for Titanic](#nn_titanic)
7. [Build an individual for DE](#individual)
8. [Differential Evolution for Titanic](#de_titanic)
9. [Understanding the evolution of hyperparameters](#de_hyperpar_evolution)
10. [Hyperparameter evolution for digits and flowers dataset](#de_digits_and_flowers)
11. [Conclusion](#conclusion)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import FixedLocator, MaxNLocator
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='html5')

from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# What is Differential Evolution? <a class="anchor" id="differential_evolution"></a>

* An evolutionary algorithm based on a popultion of models/individuals.
* It's able to find global optima in optimisation landscapes with many local minima.
* It recombines one individual of the population with a new candidate **(trial)** that is obtained by computing the difference between two further individuals in the population.
* By computing these so called difference vectors the algorithm is able to self-adapt the stepsize for exploiting the parameter space of the optimisation.  
* It can also be applied in settings where there is no optimisation function at all but some kind of feedback how good the result is.
* It can also deal with optimisation landscapes that change over time.

# How does it work? <a class="anchor" id="how"></a>

Let's directly use an example by tuning the hyperparameters of neural networks.

* For this kind of optimisation the function is unknown. We have no mathematical equation that tells us how the scores or error losses behave when we change the learning rate or increase the dropout rate etc.. 
* In this problem setup, each set of parameters we could chose for a neural network is one individual. When we initialize different parameter sets then we obtain our first population.

Now the following recipe has to be followed! ;-)

1. Generate mutant using three different individuals $p_{i}$:

    * $ \vec{m} = \vec{p}_{A} - s \cdot \left( \vec{p}_{B}-\vec{p}_{C} \right)$
2. Compute a trial individual $\vec{c}$ by doing crossover using the mutant $\vec{m}$ and a randomly selected target individual $\vec{t}$:

    * $ \vec{c} = copy \left( \vec{t} \right) $
    * Randomly select elements $c_{i}$ for being replaced with the mutant element $m_{i}$:
   
    $$c_{i}=\begin{cases}
  m_{i},  & \text{if } r R\\
  c_{i}, & \text{else }
\end{cases}$$
    
    
3. Compute losses of trials and parents.
4. Perform selection by comparing trials and parents and choosing the best for the next generation.
5. Redo until stopping criertion has been reached. 

# Code to built the model <a class="anchor" id="de_model"></a>

In [ ]:
class DifferentialEvolution:

    def __init__(self, scaling_rate, crossover_rate, population_size):
        self.scaling_rate = scaling_rate
        self.crossover_rate = crossover_rate
        self.population_size = population_size
        self.ranges = []
        
        self.generation_counter = 0
        
        self.all_generations = []
        self.min_std_criterion = 0.001
    
    def set_range(self, paramkey, range):
        self.ranges.append([paramkey, range])
    
    def set_objective(self, objective):
        self.objective = objective
    
    def init_generation(self):
        self.n_params = len(self.ranges)
        self.generation = np.zeros(shape=(self.population_size, self.n_params))
        for p in range(self.n_params):
            low = self.ranges[p][1][0]
            high = self.ranges[p][1][1]
            self.generation[:, p] = np.random.uniform(low, high, size=self.population_size)
    
    def get_trials(self):
        self.trials = np.zeros(shape=(self.population_size, self.n_params))
        for i in range(self.population_size):
            target = self.generation[i, :]
            mutant = self.mutate(i)
            self.trials[i, :] = self.crossover(mutant, target)
    
    def get_candidates(self, i):
        to_select = list(np.arange(self.population_size))
        to_select.remove(i)
        candidates = np.random.choice(to_select, 3, replace=False)
        return candidates
    
    def mutate(self, i):
        candidates = self.get_candidates(i)
        difference_vector = self.generation[candidates[1]] - self.generation[candidates[2]]
        mutant = self.generation[candidates[0]] + self.scaling_rate * difference_vector
        return mutant
    
    def crossover(self, mutant, target):
        crossover_units = np.random.uniform(0, 1, self.n_params)
        trial = np.copy(target)
        random_parameter = np.random.choice(self.n_params)
        for param in range(self.n_params):
            if crossover_units[param] <= self.crossover_rate or param == random_parameter:
                trial[param] = mutant[param]
        return trial
    
    def select(self, generation_costs, trials_costs):
        idx = np.where(trials_costs < generation_costs)[0]
        for i in idx:
            self.generation[i, :] = self.trials[i, :]

    def compute_cost(self):
        generation_costs = self.objective(self.generation)
        trials_cost = self.objective(self.trials)
        return generation_costs, trials_cost
    
    def evolve(self):
        self.init_generation()
        self.all_generations.append(self.generation)
        self.best_solutions = []
        gen_costs = self.objective(self.generation)
        while ((np.std(gen_costs) > self.min_std_criterion) and (self.generation_counter < 200)):
            self.get_trials()
            gen_costs, trials_costs = self.compute_cost()
            self.select(gen_costs, trials_costs)
            self.all_generations.append(np.copy(self.generation))
            self.generation_counter += 1
            self.best_solutions.append(np.min(gen_costs))
        print("Stopped at generation {}".format(self.generation_counter))

# Some simple examples <a class="anchor" id="simple_examples"></a>

Let's get a first feeling about how this algorithm works and abouts its performance by searching for optima in the parameter space of:

* the rosenbrock function (coming soon)
* the griewank function
* the himmelblau function

Just use an example of your choice and watch Differential Evolution during optimisation.

In [ ]:
example = "griewank"

In [ ]:
def rosenbrock(w, a=1, b=100):
    return np.power(a-w[:,0],2)+b*np.power(w[:,1]-w[:,0]**2, 2)

def griewank(w):
    sum_part = np.zeros(w.shape[0])
    cos_part = np.ones(w.shape[0])
    for i in range(w.shape[1]):
        sum_part += np.power(w[:, i], 2)
        cos_part *= np.cos(w[:, i] / np.sqrt(i+1))
    return np.ones(w.shape[0]) + 1/4000 * sum_part - cos_part


def himmelblau(w):
    part1 = np.power(np.power(w[:, 0], 2) + w[:, 1] - 11, 2)
    part2 = np.power(w[:, 0] + np.power(w[:, 1], 2) - 7, 2)
    return part1 + part2    


examples = {"rosenbrock": {"objective": rosenbrock, "range": [-2,3], "delta": 0.1}, 
              "himmelblau": {"objective": himmelblau, "range": [-6, 6], "delta": 0.1},
              "griewank": {"objective": griewank, "range": [-50, 50], "delta": 0.5}}

In [ ]:
class ShowEvolution:

    def __init__(self, example):
        self.model = DifferentialEvolution(scaling_rate=0.7, crossover_rate=0.6, population_size=15)
        self.example = example
        self.range = examples[example]["range"]
        self.objective = examples[example]["objective"]
        self.delta = examples[example]["delta"]
        self.model.set_objective(self.objective)
        self.model.set_range("w1", self.range)
        self.model.set_range("w2", self.range)
        self.model.evolve()

        self.dx, self.dy = self.delta, self.delta

        tau = np.mgrid[slice(self.range[0], self.range[1] + self.dy, self.dy),
                       slice(self.range[0], self.range[1] + self.dx, self.dx)]

        self.X = tau[0, :-1, :-1]
        self.Y = tau[1, :-1, :-1]
        self.Z = np.zeros(shape=(tau.shape[1], tau.shape[2]))
        
        for m in range(tau.shape[1]):
            for n in range(tau.shape[2]):
                self.Z[n, m] = self.objective(tau[:, n, m].reshape(1, -1))

    def show(self):
        self.fig, self.ax = plt.subplots()
        self.make_nice_contours(self.ax, self.fig)

        self.ani = animation.FuncAnimation(self.fig, self.update,
                                           init_func=self.setup_plot,
                                           frames=np.arange(len(self.model.all_generations)),
                                           interval=50,
                                           blit=True,
                                           repeat=False)
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
        self.ani.save('example.mp4', writer=writer)
        plt.close()
        return self.ani
        
        
    def make_nice_contours(self, ax, fig):
        Z = self.Z[:-1, :-1]
        
        if self.example == "griewank":
            levels = MaxNLocator(nbins=50).tick_values(Z.min(), Z.max())
            
        else:
            levels = FixedLocator([0.05, 0.1, 0.25, 0.5, 1, 2.5, 5 , 7.5,
                                   10, 20, 40, 80, 160, 320, 640, 1280]).tick_values(vmin=0, vmax=1280)
            
        cmap = plt.get_cmap('coolwarm')
        norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

        cf = ax.contourf(self.X + self.dx / 2., self.Y + self.dy / 2., Z,
                         levels=levels,
                         cmap=cmap,
                         norm=norm)
        ax.set_xlabel("$p_{1}$")
        ax.set_ylabel("$p_{2}$")
        return plt
    
    def update(self, i):
        """Update the scatter plot."""
        data = self.model.all_generations[i]
        self.scat.set_offsets(data)
        return self.scat,

    def setup_plot(self):
        data = self.model.all_generations[0]
        x = data[:, 0]
        y = data[:, 1]
        self.scat = self.ax.scatter(x, y, c="black", marker='o', s=15)
        self.ax.set_xlim([self.range[0], self.range[1]])
        self.ax.set_ylim([self.range[0], self.range[1]])
        return self.scat,

In [ ]:
gwc = ShowEvolution(example)
gwc.show()

# Prepare the Titanic <a class="anchor" id="prepare_titanic"></a>

Ok, let's start simple by building a feedforward neural network to classify the survival of the titanic. As I don't want to win this competition but rather to show what you can do with Differential Evolution I like to reduce the feature space by only using:

* Pclass
* Sex
* Age
* Embarked
* Fare
* Family size

It should be sufficient. 

In [ ]:
titanic = pd.read_csv("../input/titanic/train.csv")
titanic.head()

We have to be careful with the fare as it is a price per group/family and not per person. You can see that every family member given by the lastname and the ticket has the same fare (std of 0):

In [ ]:
titanic.loc[:, "lastname"] = titanic.Name.str.split(",").apply(lambda l: l[0])
titanic.groupby(["lastname","Ticket"]).Fare.std().fillna(0).max()

Now we can compute the price per person which should work better together with features like Pclass. Otherwise we would conclude for big families that a high price corresponds to a higher Pclass.

In [ ]:
price_map = titanic.groupby(["lastname", "Ticket"]).Fare.min() / titanic.groupby(["lastname", "Ticket"]).size()
family_size_map = titanic.groupby(["lastname", "Ticket"]).size()
titanic = titanic.set_index(["lastname", "Ticket"], drop=True)
titanic.loc[:, "price"] = titanic.index.map(price_map)
titanic.loc[:, "family_size"] = titanic.index.map(family_size_map)
titanic = titanic.reset_index()

Ok, let's get one-hot encoded values for Pclass, Embarked and Sex:

In [ ]:
titanic = pd.get_dummies(titanic, columns=["Pclass", "Embarked", "Sex"])

And fill missing ages with median:

In [ ]:
titanic.Age = titanic.Age.fillna(titanic.Age.median())

Furthermore we need to transform Age and price such that they look more normally distributed. The gradients of neural networks are direclty influenced by the features themselves and can be misled when outliers are present.

In [ ]:
fig, ax = plt.subplots(1,4,figsize=(20,5))
sns.distplot(titanic.Age, ax=ax[0], color="Orange")
sns.distplot(titanic.Age.apply(lambda l: np.log(l+20)), ax=ax[1], color="Orange")
sns.distplot(titanic.price, ax=ax[2], color="Purple")
sns.distplot(titanic.price.apply(lambda l: np.log(l+0.5)), ax=ax[3], color="Purple");

In [ ]:
titanic.Age = titanic.Age.apply(lambda l: np.log(l+20))
titanic.price = titanic.price.apply(lambda l: np.log(l+0.5))

In [ ]:
to_drop = ["PassengerId", "Ticket", "lastname", "Sex_male", "Name", "SibSp", "Parch", "Cabin", "Fare"]
titanic = titanic.drop(to_drop, axis=1)
titanic.head()

Ok, that's it! :-) Let's now build a simple neural network and define the hyperparameters we like to tune using Differential Evolution.

In [ ]:
num_titanic_features=titanic.drop("Survived", axis=1).shape[1]
num_titanic_features

# Build the NNs for Titanic <a class="anchor" id="nn_titanic"></a>

Ok, we have optimised some test optimisation functions to see how differential evolution solves the task. Now I like to explore how it will solve hyperparameter tuning for a simple feedforward neural network. I'm especially interested in:

* The number of neurons compared to regularisation given by dropout rates or weight regularisation.
* The number of epochs compared to the learning rate.

Let's go! :-)

In [ ]:
def build_titanic_nn(num_neurons_1, dropout_1,
                     num_neurons_2, dropout_2,
                     input_shape=(num_titanic_features,)):
    
    model = keras.Sequential()
    model.add(keras.Input(shape=input_shape))
    model.add(keras.layers.Dense(num_neurons_1, activation="relu"))
    model.add(keras.layers.Dropout(dropout_1))
    model.add(keras.layers.Dense(num_neurons_2, activation="relu"))
    model.add(keras.layers.Dropout(dropout_2))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    return model

# Build an individual for DE <a class="anchor" id="individual"></a>



Before we can use Differential Evolution to search for the best neural network hyperparameters, we need a class for individual neural networks that captures the logic for training and evaluating each net and for storing its parameters:

In [ ]:
from collections import OrderedDict
import copy

class Individual_Titanic:
    
    def __init__(self, params):
        self.parameters = copy.deepcopy(params)
        self.history = None
    
    def init_current(self, r_state):
        next_state = 0
        for par in self.parameters.keys():
            state = r_state + next_state
            if self.parameters[par]["val_range"].dtype == np.int:
                self.parameters[par]["current"] = np.random.RandomState(state).randint(
                    low=self.parameters[par]["val_range"][0],
                    high=self.parameters[par]["val_range"][1]) 
            else:
                self.parameters[par]["current"] = np.random.RandomState(state).uniform(
                    low=self.parameters[par]["val_range"][0],
                    high=self.parameters[par]["val_range"][1])
            next_state += 1
            
    
    def build_model(self):
        model = build_titanic_nn(num_neurons_1=self.parameters["num_neurons_1"]["current"],
                                 dropout_1=self.parameters["dropout_1"]["current"],
                                 num_neurons_2=self.parameters["num_neurons_2"]["current"],
                                 dropout_2=self.parameters["dropout_2"]["current"])
        return model
        
        
    def train(self, x_train, y_train, x_dev, y_dev, verbose=0):
        model = self.build_model()
        model.compile(loss=keras.losses.BinaryCrossentropy(),
                      optimizer=keras.optimizers.Adam(learning_rate=self.parameters["lr"]["current"]),
                      metrics=["accuracy"])
        self.history = model.fit(x_train, y_train,
                                 batch_size=32,
                                 epochs=self.parameters["epochs"]["current"],
                                 verbose=verbose,
                                 workers=4)
        dev_loss, dev_acc = model.evaluate(x_dev, y_dev)
        self.score = dev_acc
        self.loss = dev_loss
        del model
        

In [ ]:
X = titanic.drop("Survived", axis=1).values
Y = titanic.Survived.values
x_train, x_dev, y_train, y_dev = train_test_split(X, Y, stratify=Y, random_state=0)
print(x_train.shape, x_dev.shape)

Here is a small example:

In [ ]:
parameters = {
    "dropout_1": {"val_range": np.array([0.1,0.8]).astype(np.float), "current": None},
    "dropout_2": {"val_range": np.array([0.1,0.8]).astype(np.float), "current": None},
    "num_neurons_1": {"val_range": np.array([10,300]).astype(np.int), "current": None},
    "num_neurons_2": {"val_range": np.array([10,300]).astype(np.int), "current": None},
    "lr": {"val_range": np.array([0.001,0.5]).astype(np.float), "current": None},
    "epochs": {"val_range": np.array([1,20]).astype(np.int), "current": None}
}

In [ ]:
individual = Individual_Titanic(parameters)

individual.init_current(0)
for par in individual.parameters.keys():
    print("Par {}, {}".format(par, individual.parameters[par]["current"]))
individual.train(x_train, y_train, x_dev, y_dev, verbose=0)
individual.score

Ok, now we like to know how we need to change the parameters such that the accuracy increases. 

# Differential Evolution for Titanic <a class="anchor" id="de_titanic"></a>

In [ ]:
class DE_NeuralNets:
    
    def __init__(self, X, Y,
                 scaling_rate=0.5, 
                 crossover_rate=0.5, 
                 population_size=20,
                 max_generation_steps=25):
        self.scaling_rate = scaling_rate
        self.crossover_rate = crossover_rate
        self.population_size = population_size
        
        self.generation_counter = 0
        
        self.all_generations = []
        self.max_generation_steps = max_generation_steps
        self.x_train, self.x_dev, self.y_train, self.y_dev = train_test_split(
            X, Y, stratify=Y, random_state=0)
    
    def add_params(self, params):
        for par in params.keys():
            val_range = params[par]["val_range"]
            if not isinstance(val_range, np.ndarray):
                raise TypeError("The value range must be of type list.")
            if len(val_range) != 2:
                pass
            if val_range[0] > val_range[1]:
                pass
        self.parameters = params
    
    def init_generation(self):
        self.generation = []
        self.generation_costs = []
        for i in range(self.population_size):
            individual = Individual_Titanic(self.parameters)
            random_state = i*len(self.parameters)
            individual.init_current(random_state)
            individual.train(x_train=self.x_train,
                             y_train=self.y_train,
                             x_dev=self.x_dev,
                             y_dev=self.y_dev,
                             verbose=0)
            self.generation_costs.append(individual.score)
            self.generation.append(individual)
    
    def get_trials(self):
        print("-- Trials are computed --")
        self.trials = []
        self.trials_costs = []
        for i in range(self.population_size):
            target = self.generation[i]
            mutant = self.mutate(i)
            trial = self.crossover(mutant, target)
            trial.train(x_train=self.x_train,
                             y_train=self.y_train,
                             x_dev=self.x_dev,
                             y_dev=self.y_dev,
                             verbose=0)
            self.trials_costs.append(trial.score)
            self.trials.append(trial)
    
    def mutate(self, i):
        candidates = self.get_candidates(i)
        mutant = Individual_Titanic(self.generation[candidates[0]].parameters.copy())
        for par in self.parameters.keys():
            A = self.generation[candidates[1]].parameters[par]["current"]
            B = self.generation[candidates[2]].parameters[par]["current"]
            if type(A) is float:
                mutation = mutant.parameters[par]["current"] + self.scaling_rate * (A-B)
            else:
                mutation = mutant.parameters[par]["current"] + np.int(np.round(self.scaling_rate * (A-B)))
            mutation = self.check_boundaries(mutation,
                                             mutant.parameters[par]["val_range"][0],
                                             mutant.parameters[par]["val_range"][1])
            mutant.parameters[par]["current"] = mutation
        return mutant
    
    def check_boundaries(self, mutation, low_bound, high_bound):
        if mutation < low_bound:
            mutation = low_bound
        elif mutation > high_bound:
            mutation = high_bound
        return mutation
        
    def get_candidates(self, i):
        to_select = list(np.arange(self.population_size))
        to_select.remove(i)
        candidates = np.random.choice(to_select, 3, replace=False)
        return candidates
    
    def crossover(self, mutant, target):
        param_list = list(self.parameters.keys())
        random_par = np.random.choice(param_list)
        trial = Individual_Titanic(target.parameters.copy())
        for par in param_list:
            crossover_unit = np.random.uniform(0, 1)
            if crossover_unit <= self.crossover_rate or par == random_par:
                trial.parameters[par]["current"] = mutant.parameters[par]["current"]
        return trial
    
    def select(self):
        
        idx = np.where(np.array(self.trials_costs) > np.array(self.generation_costs))[0]
        
        # fuse trials and parents to form a candidate pool
        fusion = self.trials_costs[:]
        fusion.extend(self.generation_costs[:])
        candidate_pool = self.trials[:]
        candidate_pool.extend(self.generation[:]) 
        
        # sort score (descending)
        idx = np.argsort(np.array(fusion))[::-1]
    
        # only select the best individuals of both (generation & trials)
        for i in range(len(self.generation)):
            self.generation[i] = candidate_pool[idx[i]]
            self.generation_costs[i] = fusion[idx[i]]
    
    def select_v2(self):
        idx = np.where(np.array(self.trials_costs) > np.array(self.generation_costs))[0]
        for i in idx:
            self.generation[i] = self.trials[i]
            self.generation_costs[i] = self.trials_costs[i]
    
    def evolve(self):
        self.best_solutions = []
        self.worst_solutions = []
        self.init_generation()
        self.all_generations.append(self.generation[:])
        while(self.generation_counter < self.max_generation_steps):
            self.get_trials()
            self.select()
            self.all_generations.append(self.generation[:])
            self.generation_counter += 1
            self.best_solutions.append(np.max(self.generation_costs))
            self.worst_solutions.append(np.min(self.generation_costs))

In [ ]:
eve = DE_NeuralNets(X, Y)
eve.add_params(parameters)
eve.evolve()

# How does the solution look like?

Differential evolution is able to find the global optimum for the data and model given but it is not guaranteed to converge. In our case I expect that all individuals of the updated generation improve their scores during hyperparameter search. While the worst and best accuracy score might have a big difference in the beginning I assume that they become closer during iteration:


In [ ]:
plt.plot(eve.best_solutions, 'o-', label="best")
plt.plot(eve.worst_solutions, '+-', label="worst")
plt.ylabel("Best val accuracy in generation i")
plt.xlabel("Generation i")
plt.title("Evolution of best individuals")
plt.legend()

For the titanic validation dataset and the chosen features we can see that some score above 0.8 is best.

# Understanding the evolution of hyperparameters <a class="anchor" id="de_hyperpar_evolution"></a>

One part I find most interesting in using DE for hyperparameter search is to observe the evolution of hyperparameters and their related score or loss values:

In [ ]:
plt.rcParams["animation.html"] = "jshtml" 
    
class ShowHyperparameterEvolution:

    def __init__(self, model):
        self.eve = model
        self.num_generations = len(self.eve.all_generations)
        
        self.fig, self.ax = plt.subplots(1,3,figsize=(20,5))
        
        self.ax[0].set_xlim([0, 500])
        self.ax[0].set_ylim([0, 500])
        self.ax[0].set_xlabel("Neurons in layer 1")
        self.ax[0].set_ylabel("Neurons in layer 2")
        
        self.ax[1].set_xlim([0,1])
        self.ax[1].set_ylim([0,1])
        self.ax[1].set_xlabel("Dropout layer 1")
        self.ax[1].set_ylabel("Dropout layer 2")
        
        self.ax[2].set_xlim([0,0.8])
        self.ax[2].set_ylim([0,25])
        self.ax[2].set_xlabel("Learning rate")
        self.ax[2].set_ylabel("Number of epochs")
        
        
    
    def show(self):
        self.ani = animation.FuncAnimation(self.fig, self.update,
                                           init_func=self.init_fun,
                                           frames=self.num_generations,
                                           interval=900,
                                           blit=False,
                                           repeat=False)
        #Writer = animation.writers['ffmpeg']
        #writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
        #self.ani.save('example.mp4', writer=writer)
        plt.close()
        return self.ani

    def get_data(self, generation):
        data = np.zeros(shape=(len(generation), 6))
        for i in range(len(generation)):
            data[i,0] = generation[i].parameters["num_neurons_1"]["current"]
            data[i,1] = generation[i].parameters["num_neurons_2"]["current"]
            data[i,2] = generation[i].parameters["dropout_1"]["current"]
            data[i,3] = generation[i].parameters["dropout_2"]["current"]
            data[i,4] = generation[i].parameters["lr"]["current"]
            data[i,5] = generation[i].parameters["epochs"]["current"]
        return data
        
    def update(self, i):
        """Update the scatter plot."""
        generation = self.eve.all_generations[i]
        data = self.get_data(generation)
        if i==0:
            self.scat1 = self.ax[0].scatter(data[:, 0], data[:, 1], c="mediumseagreen", marker='o', s=15) 
            self.scat2 = self.ax[1].scatter(data[:, 2], data[:, 3], c="red", marker="o", s=15)
            self.scat3 = self.ax[2].scatter(data[:, 4], data[:, 5], c="purple", marker="o", s=15)
        else:
            self.scat1.set_offsets(data[:,0:2])
            self.scat2.set_offsets(data[:,2:4])
            self.scat3.set_offsets(data[:,4:6])
        #return self.scat1,

    def init_fun(self):
        return [self.ax]

In [ ]:
gwc = ShowHyperparameterEvolution(eve)
gwc.show()

### Insights

* We can see that DE finds several solutions that are similar good with close scores. 
* Furthermore we can see an interesting pattern: 
    * The number of neurons in layer 1 seem to be more peaked than in layer 2.
    * The dropout rate of layer 1 spreads a lot while the rates of layer 2 are more dense. 
    * The number of epochs vary quite a lot! 

Probably there is a dependency between these hyperparameters that we can use to improve our optimization and for a fruitful rule of thumb. 

# Hyperparameter evolution for the digits dataset <a class="anchor" id="de_digits"></a>

**Work in progress**

In [ ]:
def build_digits_cnn(num_neurons_1, dropout_1, input_shape=(28, 28, 1)):
    model = keras.Sequential()
    model.add(keras.Input(shape=input_shape))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=2))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
    #model.add(keras.layers.MaxPooling2D(pool_size=2))
    #model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(num_neurons_1, activation="relu"))
    model.add(keras.layers.Dropout(dropout_1))
    model.add(keras.layers.Dense(10, activation="softmax"))
    return model

In [ ]:
class Individual_Digits:
    
    def __init__(self, params):
        self.parameters = copy.deepcopy(params)
        self.history = None
    
    def init_current(self, r_state):
        next_state = 0
        for par in self.parameters.keys():
            state = r_state + next_state
            if self.parameters[par]["val_range"].dtype == np.int:
                self.parameters[par]["current"] = np.random.RandomState(state).randint(
                    low=self.parameters[par]["val_range"][0],
                    high=self.parameters[par]["val_range"][1]) 
            else:
                self.parameters[par]["current"] = np.random.RandomState(state).uniform(
                    low=self.parameters[par]["val_range"][0],
                    high=self.parameters[par]["val_range"][1])
            next_state += 1
    
    def build_model(self):
        model = build_digits_cnn(num_neurons_1=self.parameters["num_neurons_1"]["current"],
                                 dropout_1=self.parameters["dropout_1"]["current"])
        return model
    
    def train(self, x_train, y_train, x_dev, y_dev, verbose=0):
        model = self.build_model()
        model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                      optimizer=keras.optimizers.Adam(learning_rate=self.parameters["lr"]["current"]),
                      metrics=["accuracy"])
        self.history = model.fit(x_train, y_train,
                                 batch_size=16*self.parameters["batchsize"]["current"],
                                 epochs=5,
                                 verbose=verbose,
                                 workers=4)
        dev_loss, dev_acc = model.evaluate(x_dev, y_dev)
        self.score = dev_acc
        self.loss = dev_loss
        del model

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

In [ ]:
parameters = {
    "dropout_1": {"val_range": np.array([0.1,0.8]).astype(np.float), "current": None},
    "num_neurons_1": {"val_range": np.array([10,300]).astype(np.int), "current": None},
    "lr": {"val_range": np.array([0.005,0.01]).astype(np.float), "current": None},
    "batchsize": {"val_range": np.array([4,8]).astype(np.int), "current": None}
}

In [ ]:
individual = Individual_Digits(parameters)

individual.init_current(2)
for par in individual.parameters.keys():
    print("Par {}, {}".format(par, individual.parameters[par]["current"]))
individual.train(train_images, train_labels, test_images, test_labels, verbose=1)
individual.score

In [ ]:
class DE_NeuralNets_Digits(DE_NeuralNets):
    
    def __init__(self,
                 scaling_rate=0.5, 
                 crossover_rate=0.5, 
                 population_size=6,
                 max_generation_steps=10):
        self.scaling_rate = scaling_rate
        self.crossover_rate = crossover_rate
        self.population_size = population_size
        
        self.generation_counter = 0
        
        self.all_generations = []
        self.max_generation_steps = max_generation_steps
        
        (train_images, self.y_train), (test_images, self.y_dev) = mnist.load_data()
        train_images = train_images.reshape((60000, 28, 28, 1))
        self.x_train = train_images.astype("float32") / 255
        test_images = test_images.reshape((10000, 28, 28, 1))
        self.x_dev = test_images.astype("float32") / 255
    
    def init_generation(self):
        self.generation = []
        self.generation_costs = []
        for i in range(self.population_size):
            individual = Individual_Digits(self.parameters)
            random_state = i*len(self.parameters)
            individual.init_current(random_state)
            individual.train(x_train=self.x_train,
                             y_train=self.y_train,
                             x_dev=self.x_dev,
                             y_dev=self.y_dev,
                             verbose=0)
            self.generation_costs.append(individual.score)
            self.generation.append(individual)
            
    def mutate(self, i):
        candidates = self.get_candidates(i)
        mutant = Individual_Digits(self.generation[candidates[0]].parameters.copy())
        for par in self.parameters.keys():
            A = self.generation[candidates[1]].parameters[par]["current"]
            B = self.generation[candidates[2]].parameters[par]["current"]
            if type(A) is float:
                mutation = mutant.parameters[par]["current"] + self.scaling_rate * (A-B)
            else:
                mutation = mutant.parameters[par]["current"] + np.int(np.round(self.scaling_rate * (A-B)))
            mutation = self.check_boundaries(mutation,
                                             mutant.parameters[par]["val_range"][0],
                                             mutant.parameters[par]["val_range"][1])
            mutant.parameters[par]["current"] = mutation
        return mutant 
    
    
    def crossover(self, mutant, target):
        param_list = list(self.parameters.keys())
        random_par = np.random.choice(param_list)
        trial = Individual_Digits(target.parameters.copy())
        for par in param_list:
            crossover_unit = np.random.uniform(0, 1)
            if crossover_unit <= self.crossover_rate or par == random_par:
                trial.parameters[par]["current"] = mutant.parameters[par]["current"]
        return trial

In [ ]:
eve = DE_NeuralNets_Digits()
eve.add_params(parameters)
eve.evolve()

In [ ]:
plt.plot(eve.best_solutions, 'o-', label="best")
plt.plot(eve.worst_solutions, '+-', label="worst")
plt.ylabel("Best val accuracy in generation i")
plt.xlabel("Generation i")
plt.title("Evolution of best individuals")
plt.legend()